<a href="https://colab.research.google.com/github/luizalvesot/documents_embeddings_gemini/blob/master/documents_embeddings_gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.8/677.8 kB 13.3 MB/s eta 0:00:00


In [4]:
#Importações e configurações iniciais
import numpy as np
import pandas as pd
import google.generativeai as genai

from google.colab import userdata
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

In [6]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [27]:
#Exemplo de embedding
title = "A próxima geração de IA para desenvolvedores e Google Workspace"
sample_text = ("Título: A próxima geração de IA para desenvolvedores e Google Workspace"
                "\n"
                "Artigo completo:\n"
                "\n"
                "Gemini API e Google AI Studio: Uma maneira acessível de explorar e criar protótipos com aplicações de IA generativa")

embeddings = genai.embed_content(model = "models/embedding-001",
                                 content = sample_text,
                                 title = title,
                                 task_type = "RETRIEVAL_DOCUMENT")

print(embeddings)

{'embedding': [0.050191857, -0.037051097, -0.019044409, 0.021003231, 0.06740932, -0.0040994524, -0.025825994, -0.016898943, 0.060917404, 0.06395007, 0.013241801, 0.0141449915, -0.04666676, -0.021959215, 0.008568223, -0.023262562, 0.020403203, -0.011015431, -0.035483286, -0.0012589331, 0.009031827, 0.0065118168, -0.029067313, -0.06604572, -0.020294823, 0.020649714, 0.0154330125, -0.038301226, -0.03437842, 0.02772093, -0.049747977, 0.0500365, -0.034499392, 0.01136, -0.03590186, -0.04140091, -0.026137765, -0.04317498, -0.0032753919, -0.0024025133, 0.005209084, -0.085087605, -0.011149047, 0.024426758, -0.0046412814, -0.021756198, 0.044463847, 0.045876212, 0.01445051, -0.060285207, 0.028205752, 0.02407988, 0.06801567, -0.041790824, -0.002953777, -0.010794445, 0.023750233, -0.032771073, 0.022061417, 0.00010103863, -0.0014788097, 0.024434738, -0.01607516, 0.053333383, 0.01988855, -0.058457788, -0.044407055, -0.0007819059, 0.009658026, 0.048190154, 0.009275901, 0.0097165285, 0.06050302, -0.064

In [28]:
#Listagem de documentos que serão buscados
DOCUMENT1 = {
    "Título": "Operação do sistema de controle climático",
    "Conteúdo": "O Googlecar tem um sistema de controle climático que permite ajustar a temperatura e o fluxo de ar no carro. Para operar o sistema de controle climático, use os botões e botões localizados no console central.  Temperatura: O botão de temperatura controla a temperatura dentro do carro. Gire o botão no sentido horário para aumentar a temperatura ou no sentido anti-horário para diminuir a temperatura. Fluxo de ar: O botão de fluxo de ar controla a quantidade de fluxo de ar dentro do carro. Gire o botão no sentido horário para aumentar o fluxo de ar ou no sentido anti-horário para diminuir o fluxo de ar. Velocidade do ventilador: O botão de velocidade do ventilador controla a velocidade do ventilador. Gire o botão no sentido horário para aumentar a velocidade do ventilador ou no sentido anti-horário para diminuir a velocidade do ventilador. Modo: O botão de modo permite que você selecione o modo desejado. Os modos disponíveis são: Auto: O carro ajustará automaticamente a temperatura e o fluxo de ar para manter um nível confortável. Cool (Frio): O carro soprará ar frio para dentro do carro. Heat: O carro soprará ar quente para dentro do carro. Defrost (Descongelamento): O carro soprará ar quente no para-brisa para descongelá-lo."}

DOCUMENT2 = {
    "Título": "Touchscreen",
    "Conteúdo": "O seu Googlecar tem uma grande tela sensível ao toque que fornece acesso a uma variedade de recursos, incluindo navegação, entretenimento e controle climático. Para usar a tela sensível ao toque, basta tocar no ícone desejado.  Por exemplo, você pode tocar no ícone \"Navigation\" (Navegação) para obter direções para o seu destino ou tocar no ícone \"Music\" (Música) para reproduzir suas músicas favoritas."}

DOCUMENT3 = {
    "Título": "Mudança de marchas",
    "Conteúdo": "Seu Googlecar tem uma transmissão automática. Para trocar as marchas, basta mover a alavanca de câmbio para a posição desejada.  Park (Estacionar): Essa posição é usada quando você está estacionado. As rodas são travadas e o carro não pode se mover. Marcha à ré: Essa posição é usada para dar ré. Neutro: Essa posição é usada quando você está parado em um semáforo ou no trânsito. O carro não está em marcha e não se moverá a menos que você pressione o pedal do acelerador. Drive (Dirigir): Essa posição é usada para dirigir para frente. Low: essa posição é usada para dirigir na neve ou em outras condições escorregadias."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [29]:
df = pd.DataFrame(documents)
df.columns = ["Titulo", "Conteudo"]
df

,Titulo,Conteudo
0,Operação do sistema de controle climático,O Googlecar tem um sistema de controle climáti...
1,Touchscreen,O seu Googlecar tem uma grande tela sensível a...
2,Mudança de marchas,Seu Googlecar tem uma transmissão automática. ...


In [30]:
model = "models/embedding-001"

In [31]:
def embed_fn(title, text):
  return genai.embed_content(model = model,
                                 content = text,
                                 title = title,
                                 task_type = "RETRIEVAL_DOCUMENT")["embedding"]

In [32]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df

,Titulo,Conteudo,Embeddings
0,Operação do sistema de controle climático,O Googlecar tem um sistema de controle climáti...,"[-0.011010795, -0.026731547, -0.036728486, 0.0..."
1,Touchscreen,O seu Googlecar tem uma grande tela sensível a...,"[0.015715627, -0.040427547, 0.011117627, 0.002..."
2,Mudança de marchas,Seu Googlecar tem uma transmissão automática. ...,"[-0.009390755, -0.022475218, -0.0024638234, 0...."


In [33]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]

In [34]:
consulta = "Como faço para trocar marchas em um carro do Google?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Seu Googlecar tem uma transmissão automática. Para trocar as marchas, basta mover a alavanca de câmbio para a posição desejada.  Park (Estacionar): Essa posição é usada quando você está estacionado. As rodas são travadas e o carro não pode se mover. Marcha à ré: Essa posição é usada para dar ré. Neutro: Essa posição é usada quando você está parado em um semáforo ou no trânsito. O carro não está em marcha e não se moverá a menos que você pressione o pedal do acelerador. Drive (Dirigir): Essa posição é usada para dirigir para frente. Low: essa posição é usada para dirigir na neve ou em outras condições escorregadias.


In [35]:
generation_config = {
  "temperature": 0,
  "candidate_count": 1
}

In [36]:
prompt = f"Reescreva esse texto de uma forma mais descontraída, sem adicionar informações que não façam parte do texto: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

**Guia do Googlecar para Descolados**

Seu Googlecar é tipo um carro automático maneiro. Pra trocar as marchas, é só dar um toque na alavanca e escolher o que você quer:

* **Estacionar:** Quando você tá parado e quer que o carro fique quietinho. As rodas ficam travadas e ele não mexe nem um centímetro.
* **Ré:** Pra dar aquela ré maneirinha.
* **Neutro:** Quando você tá parado no farol ou no trânsito. O carro tá desligado e não vai se mexer até você pisar no acelerador.
* **Dirigir:** Pra acelerar pra frente.
* **Baixa:** Pra quando você tá dirigindo na neve ou em lugares escorregadios.
